# Secretary Optimizer

The secretary problem is a class of problems which works as follows:-
- We are interviewing for a new secretary.
- After each interview we must either "hire" the secretary and stop interviewing more secretaries, or "reject" the secretary and then we cannot hire them.
- We can interview at most N secretaries.
- We don't have data about the distribution of possible secretaries, but for any two secretaries we can decide which we prefer.

For such problems, the known solution which gives the best chance of hiring the best secretary is:-
- reject the first N/e secretaries
- Then, hire the first secretary which is preferable to every secretary we have interviewed so far.

We use the solution to the secretary problem to build an optimization algorithm here. For any greedy algorithm (gradient descent, hill climbing, simulated annealing with low temperature etc.) we can divide the problem into "basins". If we are anywhere in a basin, we will eventually converge to that basin's local optimum with our greedy algorithm. So, we can formulate global optimization as two problems:-

1: find the basin which contains the global optimum

2: then find the optimum within that basin

If we assume that basins are approximately flat and of equal size, we can reframe step 1 as the secretary problem: we "interview" basins in a random order by dividing the space with grid search then sampling randomly, rejecting the first N/e, then we "hire" the first basin which is preferable to all the others. Once the basin is "hired", we use a greedy algorithm (here, simulated annealing with low temperature) to find the local optimum within that basin.

In [1]:
import numpy as np
import random as r

## Griewank

Griewank is an interesting optimization problem due to its many local optima and variable dimensions. One interesting area in optimization is how well algorithms perform when there are a large number of dimensions and few objective function calls. Thus, we test 100-dimensional Griewank with 1000 function calls. For more on Griewank: https://www.sfu.ca/~ssurjano/griewank.html

In [2]:
class Griewank:
    def sample(self, x):
        prodTerm = 1
        for i in range(len(x)):
            prodTerm *= np.cos(x[i] / np.sqrt(i + 1))
        sumTerm = 0
        for i in range(len(x)):
            sumTerm += (x[i] ** 2) / 4000
        return(sumTerm - prodTerm + 1)

    def range(self):
        return (-600, 600)
    
maxFunctionCalls = 1000

## Random Search

Random Search (RS) is computationally cheap and a useful control. If your algorithm doesn't outperform RS, it's back to the drawing board!

In [3]:
class RandomSearch:
        
    def generateRandomHypothesis(self, nDimensions, ran):
        output = []
        for i in range(nDimensions):
            output.append(r.uniform(ran[0], ran[1]))
        return(output)
    
    def optimize(self,
                objectiveFunction,
                nDimensions,
                numIterations = maxFunctionCalls):
        bestX = None
        bestY = -float("inf")
        for i in range(numIterations):
            newX = self.generateRandomHypothesis(nDimensions, objectiveFunction.range())
            newY = objectiveFunction.sample(newX)
            if newY > bestY:
                bestX = newX
                bestY = newY
        return(bestX, bestY)
rs = RandomSearch()
griewank = Griewank()
bestX, bestY = rs.optimize(griewank, 10)
print(bestY)

585.9281134133468


## Simulated Annealing

Since we need to code Simulated Annealing (SA) for the Secretary Optimizer anyway, we may as well see how it performs on its own.

In [4]:
class SimulatedAnnealing:
    
    def optimize(self, objectiveFunction, nDimensions, numIterations = maxFunctionCalls, standardDeviation = 0.1, coolingFactor = 0.9):
        currentX = [np.mean(objectiveFunction.range()) for x in range(nDimensions)]
        currentY = objectiveFunction.sample(currentX)
        heat = 1
        for i in range(numIterations):
            heat = heat * coolingFactor
            newX = [r.gauss(x, standardDeviation) for x in currentX]
            newY = objectiveFunction.sample(newX)
            if newY > currentY:
                currentY = newY
                currentX = newX
            else:
                prob = r.uniform(0, 1)
                if prob > np.e ** -(currentY - newY)/(heat + (10 ** -10)):
                    currentY = newY
                    currentX = newX
        return(currentX, currentY)

sa = SimulatedAnnealing()
bestX, bestY = sa.optimize(griewank, 10)
print(bestY)

2.0001032880802168


## Secretary Optimizer

Here's the fun part, where we test Secretary Optimization and compare to the other algorithms.

In [5]:
class SecretaryOptimizer:
    def __init__(
        self,
        greedy
    ):
        self.greedy = greedy

    #Required for distributing the points for the initial interview phase
    def generateGrid(self, functionRange, nDimesnions, nPoints):
        low, high = functionRange
        points = np.zeros((nPoints, nDimesnions))
        steps = np.ceil(nPoints ** (1 / nDimesnions)).astype(int)
        for dim in range(nDimesnions):
            linspace = np.linspace(low, high, steps)
            repeats = np.ceil(nPoints / steps).astype(int)
            values = np.tile(linspace, repeats)[:nPoints]
            np.random.shuffle(values)
            points[:, dim] = values
        return(points)
        
    def optimize(self, objectiveFunction, nDimensions, numIterations = maxFunctionCalls):
        grid = self.generateGrid(objectiveFunction.range(), nDimensions, numIterations)
        np.random.shuffle(grid)
        bestY = float("inf")
        #"Reject" the first N/e secretaries
        for i in range(int(numIterations/np.e)):
            thisX = grid[i]
            thisY = objectiveFunction.sample(thisX)
            if thisY < bestY:
                bestY = thisY
                bestX = thisX
        #"Hire" the first secretary which is better than all these
        j = int(numIterations/np.e)
        exitCondition = False
        while j < numIterations - 1 and not exitCondition:
            #Keep checking secretaries until we find a better one
            j += 1
            thisX = grid[j]
            thisY = objectiveFunction.sample(thisX)
            if thisY < bestY:
                bestY = thisY
                bestX = thisX
                exitCondition = True
        #If we hit exitCondition, we behave like a greedy algorithm - in this case, simulated annealing with low temperature
        bestX = self.greedy.optimize(objectiveFunction, nDimensions, numIterations - j)
        return(bestX)

secretary = SecretaryOptimizer(sa)
bestX, bestY = secretary.optimize(griewank, 10, 1000)
print(bestY)

0.012434880440789886
